In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import weibull_min
from scipy.optimize import minimize

# (1) Modelagem de Dados de Sobrevivência sujeitos à Censura Intervalar

## (1.1) Exemplo do Livro

In [2]:
# Caminho do arquivo
url = "https://docs.ufpr.br/~giolo/asa/dados/breast.txt"

# Leitura do arquivo
dados = pd.read_csv(url, delim_whitespace=True)

# Info do Data Frame
dados.info()

C:\Users\user\AppData\Local\Temp\ipykernel_30880\3288520303.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dados = pd.read_csv(url, delim_whitespace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   left    94 non-null     int64  
 1   right   56 non-null     float64
 2   ther    94 non-null     int64  
 3   cens    94 non-null     int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 KB


In [3]:
# Visualizar: Começo
dados.head()

,left,right,ther,cens
0,0,7.0,1,1
1,0,8.0,1,1
2,0,5.0,1,1
3,4,11.0,1,1
4,5,12.0,1,1


In [4]:
# Visualizar: Fim
dados.tail()

,left,right,ther,cens
89,34,NaN,0,0
90,35,NaN,0,0
91,35,39.0,0,1
92,44,48.0,0,1
93,48,NaN,0,0


In [5]:
# Ajuste nos Dados
df = dados.copy()
df["left"] = df["left"].replace(0, 0.01)
df["right"] = df["right"].replace(np.nan, np.inf)

In [6]:
# Visualizar: Começo
df.head()

,left,right,ther,cens
0,0.01,7.0,1,1
1,0.01,8.0,1,1
2,0.01,5.0,1,1
3,4.00,11.0,1,1
4,5.00,12.0,1,1


In [7]:
# Visualizar: Fim
df.tail()

,left,right,ther,cens
89,34.0,inf,0,0
90,35.0,inf,0,0
91,35.0,39.0,0,1
92,44.0,48.0,0,1
93,48.0,inf,0,0


### (3.1.1) Ajuste via Modelo Weibulll

In [8]:
# Função para o Tempo de Falha não Censurado
def surv_interval(L, R, x_cov, coeffs_par, shape_par, scale_par):
    # Preditor Linear
    effect = np.dot(x_cov, coeffs_par)

    # Contribuição para Função Verossimilhança
    surv_L = 1 - weibull_min.cdf(L / np.exp(effect), shape_par, scale=scale_par)
    surv_R = 1 - weibull_min.cdf(R / np.exp(effect), shape_par, scale=scale_par)

    return surv_L - surv_R

# Função para o Tempo de Falha Censurado
def surv_cens(L, x_cov, coeffs_par, shape_par, scale_par):
    # Preditor Linear
    effect = np.dot(x_cov, coeffs_par)

    # Contribuição para Função Verossimilhança
    surv_L = 1 - weibull_min.cdf(L / np.exp(effect), shape_par, scale=scale_par)

    return surv_L

# Função Log-verossimilhança
def loglikelihood(par, interval, cens, X_cov):
    # Número de parâmetros
    n_par = len(par)

    # Distinção de parâmetros
    shape_par = par[0]
    scale_par = np.exp(par[1])
    coeffs_par = par[2:n_par]

    # Distinção dos Limites
    L = interval[:, 0]
    R = interval[:, 1]

    # Função Log-verossimilhança
    S1 = surv_interval(L=L, R=R, x_cov=X_cov, coeffs_par=coeffs_par,
                       shape_par=shape_par, scale_par=scale_par)
    S2 = surv_cens(L=L, x_cov=X_cov, coeffs_par=coeffs_par,
                   shape_par=shape_par, scale_par=scale_par)

    flv = np.sum(cens * np.log(S1) + (1 - cens) * np.log(S2))

    return -flv

In [9]:
# Separação dos Dados
y =  np.array([df["left"], df["right"]]).T # Variável Resposta
X = np.array([df["ther"]]).T               # Matrix de Covariáveis

In [10]:
# Número de parâmetros para estimar
n_par = 2 + X.shape[1]

# Chute Inicial
init = np.ones(n_par)

# Minimização
ajust = minimize(
    loglikelihood,
    init,
    args=(y, dados["cens"], X),
    method="BFGS",
)

print(ajust)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 143.03320037310212
        x: [ 1.614e+00  3.331e+00  5.680e-01]
      nit: 15
      jac: [-5.722e-06 -3.815e-06  1.907e-06]
 hess_inv: [[ 3.746e-02 -5.072e-04 -6.809e-03]
            [-5.072e-04  1.135e-02 -1.129e-02]
            [-6.809e-03 -1.129e-02  3.111e-02]]
     nfev: 72
     njev: 18


### (3.1.2) Ajuste via Modelo Exponencial por Partes de Potência ($MEPP$)